# Deforestation Mapping in Hidalgo Mexico

We want to map deforestation areas from Sentinel-1 data. 
Our testsite is a mountainious region in Hidalgo Mexico which is dominated by temperate forest.o
This is the underlying analysis notebook.



## Package Import

We are importing all relevant packages into our namespace

In [1]:
using Pkg
Pkg.activate(".")

"/home/crem_fe/Daten/esdl_call/Project.toml"

In [2]:
using Revise

In [3]:
using ESDL

In [12]:
using ESDLPlots

┌ Info: Recompiling stale cache file /home/crem_fe/.julia/compiled/v1.1/ESDLPlots/keLbn.ji for ESDLPlots [d555b242-3f29-57aa-84ea-3df92a135dfd]
└ @ Base loading.jl:1184


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-79420671181482545\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-10848236172076318108\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/assetserver/72031935c082d03c0c3ed3153fb16aaf3542b0d7-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-79420671181482545\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

In [4]:
using Distributed

In [19]:
addprocs(6)
@everywhere begin
    using Pkg
    Pkg.activate(".")
end

In [8]:
@everywhere using Revise

In [9]:
using ESDL

In [20]:
@everywhere using ESDL
#using ESDLPlots

In [21]:
@everywhere using Dates: Date
@everywhere using Statistics: mean, median, quantile

In [22]:
@everywhere using RecurrenceAnalysis

In [7]:
@everywhere function copyfun(xout, xin, inds)
    xout .= xin[inds]
end

function copyfun2D(xout, xin, inds)
    xout .= xin[inds[1], inds[2]]
end



copyfun2D (generic function with 1 method)

## Helper function

We are going to first define some helper functions which are needed for the access of the data cube.

In [8]:
function restrainCube(cubedata)
    hasvalues = mapslices(x -> any(!ismissing, x), cubedata, ("Lon", "Lat", "Var"), max_cache=1e8)

    inds = findall(!ismissing, hasvalues[:])
    newtimeaxis = TimeAxis(cubedata.timeAxis.values[inds])

    # remap to new Time Axis
    indims = InDims("Time")
    outdims = OutDims(newtimeaxis)
    # NOTE: inds is an argument for copyfun
    newcube = mapCube(copyfun, cubedata, inds, indims=indims, outdims=outdims, max_cache=1e8)
end



restrainCube (generic function with 1 method)

In [11]:
c = Cube("/home/crem_fe/Daten/esdl_call/generation/cubes/s1cube_1dayallzarrbbnew/")

Collection of ZArray Cube with the following dimensions
Lon                 Axis with 1071 Elements from -98.6870371374164 to -98.49479766661483
Lat                 Axis with 1008 Elements from 20.69170568459913 to 20.51078498637746
Time                Axis with 915 Elements from 2014-01-02 to 2019-01-01
Variable            Axis with 4 elements: sentinel1_vh_d sentinel1_vv_a sentinel1_vv_d sentinel1_vh_a 
Total size: 18.4 GB


In [13]:
data= subsetcube(c, time=(Date(2017,3,1),Date(2019,3,1)))

Collection of ZArray Cube with the following dimensions
Lon                 Axis with 1071 Elements from -98.6870371374164 to -98.49479766661483
Lat                 Axis with 1008 Elements from 20.69170568459913 to 20.51078498637746
Time                Axis with 337 Elements from 2017-03-01 to 2019-01-01
Variable            Axis with 4 elements: sentinel1_vh_d sentinel1_vv_a sentinel1_vv_d sentinel1_vh_a 
Total size: 6.78 GB


In [23]:
@everywhere function pixeltrend(pix_trend, pix, dist)
    #@show dist
    #ind = findall(!ismissing, pix)
    rp = RecurrenceMatrix(collect(skipmissing(pix)), dist)
    pix_trend .= trend(rp)
end

In [15]:
indims=InDims("Time")
outdims=OutDims()

OutDims((), (), zero, identity, :auto, false, AsArray(), 1)

In [24]:
@everywhere dB(x) = 10 * log10(x)

In [25]:
cube_dB = map(dB, data)

Transformed cube Collection of ZArray Cube with the following dimensions
Lon                 Axis with 1071 Elements from -98.6870371374164 to -98.49479766661483
Lat                 Axis with 1008 Elements from 20.69170568459913 to 20.51078498637746
Time                Axis with 337 Elements from 2017-03-01 to 2019-01-01
Variable            Axis with 4 elements: sentinel1_vh_d sentinel1_vv_a sentinel1_vv_d sentinel1_vh_a 
Total size: 6.78 GB


In [26]:
trend_dB = mapCube(pixeltrend, cube_dB, 1.5, indims=indims, outdims=outdims)

Progress: 100%|█████████████████████████████████████████| Time: 0:02:40m


ZArray Cube with the following dimensions
Lon                 Axis with 1071 Elements from -98.6870371374164 to -98.49479766661483
Lat                 Axis with 1008 Elements from 20.69170568459913 to 20.51078498637746
Variable            Axis with 4 elements: sentinel1_vh_d sentinel1_vv_a sentinel1_vv_d sentinel1_vh_a 
Total size: 20.59 MB


In [36]:
exportcube(trend_dB, "generation/cubes/s1cube_trend_dB.nc")

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [21]:
@everywhere function prange(pix_prange, pix) 
   ts = collect(skipmissing(pix))
   if size(ts,1) == 0
       return missing
   end
   q5, q95 = quantile(ts, [0.05, 0.95])
    pix_prange .=q95 - q5
end

In [22]:
prange_dB = mapCube(prange, cube_dB, indims=indims, outdims=outdims)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:32m


ZArray Cube with the following dimensions
Lon                 Axis with 1071 Elements from -98.6870371374164 to -98.49479766661483
Lat                 Axis with 1008 Elements from 20.69170568459913 to 20.51078498637746
Variable            Axis with 4 elements: sentinel1_vh_d sentinel1_vv_a sentinel1_vv_d sentinel1_vh_a 
Total size: 20.59 MB


In [24]:
exportcube(prange_dB, "generation/cubes/s1cube_prange_dB.nc")

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


In [23]:
prange_dB[:,:,1]

1071×1008 Array{Union{Missing, Float32},2}:
   4.54165    4.02769    3.90328  …    5.01906    4.85577    5.05365
   5.79451    4.88501    5.17838       5.37811    4.63482    4.27278
 Inf          4.9698     5.61016       5.98264    4.75773    4.85357
 Inf         11.2049     6.42753       4.99875    4.83544    6.13306
  10.7195    18.5997   Inf             4.79146    4.77906    4.58645
   7.22886    6.79005  Inf        …    4.79477    5.74951    5.31587
   4.77874    4.56957    4.57585       4.20219    4.62925    5.42513
   3.9575     4.12276    5.40755       5.80259    4.3232     4.74766
   4.34092    4.89407    6.43933       4.95928    5.04079    4.81767
   4.35751    4.43461    5.30503       4.47699    5.14818    4.54073
   3.99686    5.4978     3.69142  …    5.77741    4.77129    4.85984
   4.42309    4.63983    5.58666       4.7787     4.24466    4.36454
   5.85631    5.92925    4.60246       4.37703    4.90934    4.07065
   ⋮                              ⋱    ⋮                   

In [29]:
using Colors

In [32]:
msc = getMSC(c)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:28


ZArray Cube with the following dimensions
MSC                 Axis with 183 Elements from 1900-01-01 to 1900-12-31
Lon                 Axis with 1071 Elements from -98.6870371374164 to -98.49479766661483
Lat                 Axis with 1008 Elements from 20.69170568459913 to 20.51078498637746
Variable            Axis with 4 elements: sentinel1_vh_d sentinel1_vv_a sentinel1_vv_d sentinel1_vh_a 
Total size: 3.68 GB
